In [37]:
import nltk
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.core.display import HTML
import matplotlib.pyplot as plt
stopwords = nltk.corpus.stopwords.words('english')

In [38]:
def freq(word, tokens):
    return tokens.count(word)

def word_count(tokens):
    return len(tokens)

def tf(word, tokens):
    return (freq(word, tokens) / float(word_count(tokens)))

def getFreqByDoc(doc):
    #get tokens
    tokens = nltk.word_tokenize(str(doc[1]) + ' ' + str(doc[2]))
    
    #get bitokens
    bi_tokens =  nltk.bigrams(tokens)
    bi_tokens = [' '.join(token).lower() for token in bi_tokens]
    bi_tokens = [token for token in bi_tokens if token not in stopwords]

    tokens = [token.lower() for token in tokens if len(token) > 2]
    tokens = [token for token in tokens if token not in stopwords]

    #merge tokens and bi
    alltokens = []
    alltokens.extend(tokens)
    alltokens.extend(bi_tokens)

    '''
    olist = []
    for word in (keywords):
        f = freq(word, alltokens)
        ilist = [doc[0], word, f, doc[3], doc[5]]
        olist.append(ilist)
    df = pd.DataFrame(olist, columns=['doc','word', 'frequency', 'source', 'sample_type'])
    '''
    wc = word_count(tokens)
    olist = []
    for token in (alltokens):
        f = freq(token, alltokens)
        ilist = [doc[0], token, f, wc, f/float(wc), doc[3], doc[5]]
        olist.append(ilist)
        df = pd.DataFrame(olist, columns=['doc','word', 'frequency', 'word_count', 'tf','source', 'sample_type'])
    
    return df

def createFreqDf(sdf):
    #pass a sample df
    #make a list so that it can be iterated
    mfgDfSampleList = sdf.as_matrix().tolist()
    mfgFreqDfList = []
    for mfg in mfgDfSampleList:
        df = getFreqByDoc(mfg)
        mfgFreqDfList.append(df)

    mfgFreqDf = pd.concat(mfgFreqDfList)
    return mfgFreqDf

In [39]:
#reading the abstracts in to a DF
mfgDf = pd.DataFrame.from_csv('../../data/positive_data.csv', index_col=None)
nsfDf = pd.DataFrame.from_csv('../../data/negative_data.csv', index_col=None)

In [40]:
#draw samples
mfgDfSample = mfgDf.sample(5)
mfgDfSample["sample_type"] = "MFG"

'''nsfDfSample = nsfDf.sample(5)
nsfDfSample["sample_type"] = "NON-MFG"'''

'nsfDfSample = nsfDf.sample(5)\nnsfDfSample["sample_type"] = "NON-MFG"'

In [41]:
mfgDfSample.head()

,id,title,abstract,event_name,event_year,sample_type
2109,NSF_20140101_1447449,BIGDATA: F: DKA: CSD: Human and Machine Co-Pro...,Human experts are crucial to data analysis. Th...,NSF,2014.0,MFG
3259,NSF_20150101_1525919,III: Small: Collaborative Research: Towards In...,This research project investigates the design ...,NSF,2015.0,MFG
1301,NSF_20140101_1419210,BIGDATA: Small: DA: A Random Projection Approach,"With the advent of Internet, numerous applicat...",NSF,2014.0,MFG
1122,NSF_20130101_1350562,SBIR Phase II: WIldlife Deterrence from Hazard...,This Small Business Innovation Research (SBIR)...,NSF,2013.0,MFG
3674,CIRP_20090101_2009-58-1-0495-S,Performance of water-soluble fullerenol as nov...,We propose a novel chemical–mechanical polishi...,CIRP,2009.0,MFG


In [42]:
mfgf = createFreqDf(mfgDfSample)
#print(mfgf)
nsff = createFreqDf(nsfDfSample)
#print(nsfgf)    
mfgf.head()

,doc,word,frequency,word_count,tf,source,sample_type
0,NSF_20140101_1447449,bigdata,0,186,0.0,NSF,MFG
1,NSF_20140101_1447449,dka,0,186,0.0,NSF,MFG
2,NSF_20140101_1447449,csd,0,186,0.0,NSF,MFG
3,NSF_20140101_1447449,human,0,186,0.0,NSF,MFG
4,NSF_20140101_1447449,machine,0,186,0.0,NSF,MFG
